# data loading and initial filtering

In [1]:
import math
from data_loader import Data_loader
from pair_filter import NPD_filter
from pair_filter import OLS_filter
import matplotlib.pyplot as plt



# loading data
start_idx = 0


dl = Data_loader(num_month_train=8, num_month_test=2)
window_train, window_test = dl.get_window(start_idx=start_idx)



# NPD selection
NPD = NPD_filter(window_train)
pairs, distance = NPD.select_pairs(percentile=5, verbose=True)

# OLS selection
OLS = OLS_filter(window_train)
pairs, params = OLS.select_pairs(pairs, verbose=True, confidence_level=0.01)


35432 pairs selected
min distance 0.0
max distance 0.7758473415183631
5938 out of 35432 pairs passed the test
pass rate: 0.17


# select top-k pairs accorindg to train sharpe ratio

In [2]:
from backtest import *

top_k = 50

bt = PairTradingBacktester_TrainTest(pairs[:300], params[:300], window_train, window_test) # for testing purpose
# bt = PairTradingBacktester_TrainTest(pairs, params, window_train, window_test) # complete
summary = bt.run(top_k=top_k)


spreads_train  = bt.train_spreads_df
spreads_test  = bt.test_spreads_df
returns_test   = bt.test_returns_df
returns_train   = bt.train_returns_df
comp_weights=bt.comp_weights

# EW optimizer

In [5]:
from optimizers import EW_optimizer

optimal_weights=EW_optimizer(returns_train) # train on training data

port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data


port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

-6.5510991855781056

# Sharpe Optimizer

In [6]:
from optimizers import Sharpe_optimizer

optimal_weights=Sharpe_optimizer(returns_train)  # train on training data
port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data
port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (501 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        

In [ ]:
port_ret

date
1990-07-02    0.000000
1990-07-03    0.003723
1990-07-05   -0.002052
1990-07-06    0.003221
1990-07-09    0.012017
1990-07-10    0.003471
1990-07-11    0.012385
1990-07-12   -0.002068
1990-07-13    0.018666
1990-07-16   -0.000239
1990-07-17    0.005135
1990-07-18    0.001995
1990-07-19    0.005914
1990-07-20    0.006273
1990-07-23    0.012112
1990-07-24    0.002762
1990-07-25    0.372964
1990-07-26    0.004834
1990-07-27   -0.001771
1990-07-30    0.019225
1990-07-31    0.017632
dtype: float64